# CollateX example

Uses CollateX Python to align Slavonic prayer (Tomas Kliment Mikulka).

## Imports

In [1]:
import re
import string
import pprint
import json
from collatex import *

## Read witnesses and sigla into lists
Sigla are in *sigla.txt*, witness texts are in *readings.txt*. Both are plain-text files with information for each witness on its own line, so they can be traversed in sync by offset.

Lines 17 and 18 are apparenty duplicates (both Hil 378), so one has been removed manually.

Preprocess by replacing non-breaking space (`\u00A0`) and newline (`\n`) with space and then collapse sequences of spaces

In [2]:
with open("sigla.txt", "r") as sigla_file:
    sigla = [s.strip() for s in sigla_file.read().splitlines()]
with open("readings.txt", "r") as readings_file:
    readings = [r.strip() for r in readings_file.read().splitlines()]

## Function to tokenize witness strings

In [3]:
regexTokenize = re.compile(r'[ \t\r\n]+') 
def tokenize(inText):
    tokens = regexTokenize.split(inText)
    return [token for token in tokens if token and not re.match('^<.+?>$', token)]

## Function to perform Soundex normalization (called by `normalize()`

Better results with two characters than with four, perhaps because of abbreviations like gm (gospodinomъ)

In [4]:
def soundexify(inText):
    """Soundex normalization

    In this order
        process the manyToOne, oneToMany, and oneToOne groups
        degeminate all geminate consonants (e.g., нн > н)
        strip all non-word-initial vowels
        truncate long words to no more than four characters"""
    # http://stackoverflow.com/questions/764360/a-list-of-string-replacements-in-python
    # TODO: Should the variables be defined outside the function to avoid repeated initialization when called?
    manyToOne = [('оу', 'у'), ('шт', 'щ')]
    for k, v in manyToOne:
        inText = inText.replace(k, v)
    oneToMany = [('ѿ', 'ѡт'), ('ѯ', 'кс'), ('ѱ', 'пс')]
    for k, v in oneToMany:
        inText = inText.replace(k, v)
    # TODO: Is there a more legible way to chain the one-to-one replacements (see the end of this section)?
    # TODO: Same question as above concerning definitions.
    # ja letters
    lettersJa = 'ѧѩꙙꙝꙗя'
    lettersJaReplacement = 'ѧ' * len(lettersJa)
    transJa = str.maketrans(lettersJa, lettersJaReplacement)
    # e letters
    lettersE = 'еєѥ'
    lettersEReplacement = 'е' * len(lettersE)
    transE = str.maketrans(lettersE, lettersEReplacement)
    # i letters
    lettersI = 'ыꙑиіїꙇй'
    lettersIReplacement = 'и' * len(lettersI)
    transI = str.maketrans(lettersI, lettersIReplacement)
    # o letters
    lettersO = 'оꙩꙫꙭꙮѡꙍѽѻꚙꚛ'
    lettersOReplacement = 'о' * len(lettersO)
    transO = str.maketrans(lettersO, lettersOReplacement)
    # u letters
    lettersU = 'уꙋюꙕѵѷӱѹ'
    lettersUReplacement = 'у' * len(lettersU)
    transU = str.maketrans(lettersU, lettersUReplacement)
    # oN letters
    lettersON = 'ѫѭꙛ'
    lettersONReplacement = 'ѫ' * len(lettersON)
    transON = str.maketrans(lettersON, lettersONReplacement)
    # jat letters
    lettersJat = 'ѣꙓ'
    lettersJatReplacement = 'ѣ' * len(lettersJat)
    transJat = str.maketrans(lettersJat, lettersJatReplacement)
    # jer letters
    lettersJer = 'ьъ'
    lettersJerReplacement = 'ь' * len(lettersJer)
    transJer = str.maketrans(lettersJer, lettersJerReplacement)
    # z letters
    lettersZ = 'зꙁꙃѕꙅ'
    lettersZReplacement = 'з' * len(lettersZ)
    transZ = str.maketrans(lettersZ, lettersZReplacement)
    return regexNoninitialVowel.sub('', regexGeminate.sub(r'\1', inText.translate(transJa).translate(transE).
                                                          translate(transI).translate(transO).translate(transU).
                                                          translate(transON).translate(transJat).translate(transJer).
                                                          translate(transZ)))[0:3]

## Function to convert superscript to base

In [5]:
def unsuper(inText):
    supers = 'ⷠⷡⷢⷣⷤⷥⷦⷧⷨⷩⷪⷫⷬⷭⷮⷯⷰⷱⷲⷳⷴⷶⷷⷹⷺⷻⷼⷽⷾⷿ𞀰𞀱𞀲𞀳𞀴𞀵𞀶𞀷𞀸𞀹𞀺𞀻𞀼𞀽𞀾𞀿𞁀𞁁𞁂𞁃𞁄𞁅𞁆𞁇𞁈𞁉𞁌𞁬ꙴꙵꙶꙷꙸꙹꙺꙻꚌꚍꚎꚏᵸ'
    supersReplacement = 'ивгджзлльнопрстхцчшщѳаеꙋѣюꙗѧѫѭабвгдежзиклмопрстуфхцчшыэюіꙑєиїуъыьѡъьфѥн'
    transSupers = str.maketrans(supers, supersReplacement) 
    return inText.translate(transSupers)

## Function to normalize tokens

In [6]:
regexPunc = re.compile("[" + string.punctuation + "\u0360" + "]")
regexGeminate = re.compile(r'(.)\1')
regexNoninitialVowel = re.compile(r'\B[аеиоуѧѫѣь]')

def normalize(token):
    return soundexify(unsuper(regexPunc.sub("", token).lower()))

## Function to create JSON data

In [7]:
def processRdg(siglum, inText):
    """Returns JSON data for rdg"""
    witness = {'id': siglum, 'tokens': []}
    for token in inText:
        token = {'t': token, 'n': 'x' if len(normalize(token)) == 0 else normalize(token)}
        witness['tokens'].append(token)
    return witness

## Create complete JSON input

In [8]:
witnessData = [processRdg(sigla[i], tokenize(readings[i])) for i in range(len(sigla) - 1)]
dictionaryInput = {
    "witnesses" : witnessData
}
jsonInput = json.dumps(dictionaryInput)

## Create and view collation

In [9]:
collate(json.loads(jsonInput), segmentation=False, output="html" )

A,-,всемогꙑи,прѣвѣчнꙑи,боже,исповѣдавшоу,сѧ,тебѣ,рабоу,твоемоу,-,имрк,-,твоеѧ,-,ради,-,ч͠стотꙑ,-,грѣхꙑ,отъпоустиши,-,-,да,-,болшми,-,-,-,наоучиши,-,къ,разоумоу,твоѥмоу,и,наставиши,-,отъ,моукꙑ,-,на,помощь,-,къ,твоеи,милости,преити,-,-,-,-,г͠мь,нашимь,и͠с,х͠мь,с͠номь,твоимь,-,с,нимже,живеши,и,ц͠рьствоуеши,боже,съ,-,единениемъ,-,с͠тго,д͠ха,-,н͠нѣ,-,и,присно,и,в,-,в.,-,-,-
B,-,всемогїи,прѣвѣчнꙑи,боже,исповѣдавшоу,сѧ,тебѣ,рабоу,твоемоу,-,имрк,-,-,-,-,-,-,-,грѣхꙑ,отпоустиши,-,-,да,-,болшми,-,-,-,наоучиши,-,к,разоумоу,твоемоу,и,наставиши,-,от,моукꙑ,-,на,помощь,-,к,твоеи,милости,преити,-,-,-,-,г͠мь,нашимъ,и͠с,х͠омъ,с͠номь,твоимь,-,с,нимже,живеши,и,ц͠рьствоуеши,боже,съ,-,единениемъ,-,с͠тго,д͠ха,-,н͠нѣ,-,и,присно,и,в,-,-,вѣкꙑ,вѣкомъ,аминь
L,-,всемогꙑи,прѣвѣчнꙑи,б͠же,исповѣдавшоу,сѧ,тебѣ,рабоу,твоемоу,-,имрк,-,твоеѧ,-,ради,-,чистотꙑ,-,грѣхꙑ,ѿпоустиши,-,-,да,-,болшимъ,-,-,-,наоучиши,-,к,разоумоу,твоемоу,и,наставиши,-,ѿ,моукꙑ,-,на,помощь,-,къ,твоеи,милости,преити,-,-,-,-,г͠мь,нашимъ,и͠с,х͠омъ,с͠номъ,твоим,-,с,нимже,живеши,и,ц͠рьствоуеши,б͠же,съ,-,едіненіемъ,-,с͠тго,д͠ха,-,н͠нѣ,-,и,присно,и,в,-,-,вѣкꙑ,вѣкомъ,аминь
АА,-,всемогѫи,вѣчнꙑи,б͠е,исповѣдавшю,сѧ,тебѣ,рабоу,твоемоу,-,имрк,-,твоеѧ,-,ради,-,чистотꙑ,-,грѣхꙑ,ѿпоустиши,-,-,да,-,болшимъ,-,-,-,наоучушу,-,к,разоумоу,твоемоу,и,наставиши,-,ѿ,моукꙑ,-,на,помощь,-,къ,твоеи,м͠лти,преити,-,-,-,-,г͠мь,нашимь,и͠с,х͠мь,с͠номь,твоимь,-,с,нимже,живеши,и,ц͠рьствоуеши,боже,съ,-,единениемъ,-,с͠тго,д͠ха,-,н͠нѣ,-,и,пр͠сно,и,во,-,-,вѣкꙑ,-,-
X,-,всемогꙑи,превѣчнꙑи,б͠е,исповѣдавшоу,сѧ,тебѣ,рабоу,твоемоу,-,имрк,-,твоеѧ,-,ради,-,ч͠стотꙑ,-,грѣхꙑ,ѿпоусти,емоу,-,да,-,болши,-,-,-,наоучиши,его,к,разоумоу,твоемоу,и,наставиши,его,ѿ,моукꙑ,свободити,сѧ,-,и,кь,твоеи,м͠лти,пріити,-,помощію,и,б͠лгодатію,единороднаго,с͠на,-,г͠а,н͠шего,и͠са,х͠са,с,нимже,живеши,и,ц͠рьствоуеши,б͠же,со-,-,единеніемь,-,с͠тго,д͠ха,-,н͠нѣ,-,и,п͠рсно,и,во,-,-,вѣкꙑ,вѣкомь,аминь
Z,-,всемогꙑи,превѣчнꙑи,б͠же,исповѣдавшоу,сѧ,тебѣ,рабоу,твоемоу,-,имрк,-,твоеѧ,-,ради,-,чистотꙑ,-,грѣхꙑ,ѿпоусти,емоꙑ,-,да,-,болши,-,-,-,наоучиши,его,к,разоумоу,твоемоу,и,наставиши,его,ѿ,моукꙑ,свободити,сѧ,-,и,ко,твоеи,м͠лти,приити,-,помощію,и,б͠лгодатію,единороднаго,с͠на,твоего,г͠а,н͠шего,и͠са,͠хса,с,нимже,живеши,и,ц͠рьствоуеши,б͠же,со-,-,единениемь,-,с͠тго,д͠ха,-,н͠нѣ,-,и,пр͠сно,и,во,-,-,вѣкꙑ,вѣкомь,-
I,владꙑко,всемогаи,вѣчнꙑи,б͠е,исповѣдавшоу,сѧ,тебѣ,рабоу,твоемоу,семоу,имрк,-,твоеѧ,-,ради,-,ч͠стотꙑ,-,-,ѿпоусти,емоу,грѣхꙑ,да,ѡ,болшемъ,наставиши,его,и,наоучиши,его,в,разоум,-,и,-,-,отъ,мѫкꙑ,-,на,помощь,-,-,твоеи,милости,преити,не,презри,но,помилоуи,г͠мь,нашимъ,і͠сь,х͠мь,с͠нмь,твоимь,единороднꙑмъ,с,нимже,живеши,и,ц͠рьствоуеши,-,-,въ,ѥдинении,-,с͠тго,д͠ха,-,н͠нѣ,-,и,присно,и,вѣ,-,-,вѣк,-,-
W,владꙑко,всемогаи,вѣчнꙑи,б͠е,исповѣдавшоу,сѧ,тебѣ,рабоу,твоемоу,семоу,имрк,-,твоеа,-,ради,-,чистотꙑ,-,-,ѿпоусти,емоу,грѣхꙑ,да,ѡ,болшемь,наставиши,-,и,наоучиши,его,вь,разоумь,-,и,-,-,ѿ,моукꙑ,-,на,помощь,-,-,твоеи,м͠лти,приити,не,презри,но,помилоуи,г͠демь,нашимь,и͠с,х͠омь,с͠номь,твоимь,единороднꙑмъ,с,нимже,живеши,и,ц͠рьствоуеши,-,-,въ,единении,-,с͠тго,д͠ха,-,н͠нѧ,-,и,пр͠сно,-,в,-,-,вѣкꙑ,-,-
Y,вл͠ко,всесилне,вѣчнꙑи,б͠е,исповѣдавшоу,сѧ,тебѣ,рабоу,твоемоу,семоу,имрк,-,твоеа,б͠лгости,ради,-,-,-,-,ѿпоусти,емоу,грѣхꙑ,да,и,лоучшимь,наставиши,-,и,наоучиши,-,-,разоумоу,-,и,-,-,ѿ,моукꙑ,-,на,помощь,-,-,твоеи,бл͠гсти,приити,не,презри,но,помилоуи,г͠демь,нашимь,и͠с,х͠омь,с͠номь,твоимь,единороднꙑмъ,с,нимже,живеши,и,ц͠рьствоуеши,-,-,в,соединение,-,с͠тго,д͠ха,-,н͠нѣ,-,и,пр͠сно,и,въ,вѣкꙑ,-,вѣко,!,-
T,-,всемогьіи,вѣчнꙑи,б͠е,исповѣдавшоу,се,тебѣ,рабоу,твоѥмоу,-,имрк,милости,твоѥ,-,ради,-,-,-,грѣхꙑ,ѿпоустиши,-,-,да,-,болшими,наоучиши,и,и,-,-,вь,разоумь,-,-,наставиши,и,ѿ,моукꙑ,же,на,помощь,-,кь,твоѥи,масти,прити,-,-,-,-,г͠мь,нашимь,и͠с,хмь,с͠номь,твоимь,-,с,нимьже,живеши,и,ц͠рьствоуеши,б͠е,-,вь,ѥдинѥни,д͠ха,ст͠го,-,-,н͠нꙗ,-,-,-,-,-,-,-,-,-,-
C,-,всемогїи,вѣчнꙑи,боже,исповѣдающоу,сѧ,тебѣ,рабоу,твоемоу,-,имрк,-,твоею,дѣтелию,очисти,и,-,-,грѣхꙑ,отпоусти,-,-,-,-,-,-,-,и,-,-,в,разоумъ,-,-,настави,и,от,моукꙑ,же,на,помощь,-,